<a href="https://colab.research.google.com/github/Chaitanya-Atluri/CNN-from-scratch-on-MNIST/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################################################################################
#
# LOGISTICS
#
#    Your name as in eLearning
#    Your UT Dallas identifier
#
# DESCRIPTION
#
#    Image classification in PyTorch for ImageNet reduced to 100 classes and
#    down sampled such that the short side is 64 pixels and the long side is
#    >= 64 pixels
#
#    This script achieved a best accuracy of ??.??% on epoch ?? with a learning
#    rate at that point of ?.????? and time required for each epoch of ~ ??? s
#
# INSTRUCTIONS
#
#    1. Go to Google Colaboratory: https://colab.research.google.com/notebooks/welcome.ipynb
#    2. File - New Python 3 notebook
#    3. Cut and paste this file into the cell (feel free to divide into multiple cells)
#    4. Runtime - Run all
#
# NOTES
#
#    0. For a mapping of category names to directory names see:
#       https://gist.github.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57
#
#    1. The original 2012 ImageNet images are down sampled such that their short
#       side is 64 pixels (the other side is >= 64 pixels) and only 100 of the
#       original 1000 classes are kept.
#
#    2. Build and train a RegNetX image classifier modified as follows:
#
#       - Set stride = 1 (instead of stride = 2) in the stem
#       - Replace the first stride = 2 down sampling building block in the
#         original network by a stride = 1 normal building block
#       - The fully connected layer in the decoder outputs 100 classes instead
#         of 1000 classes
#
#       The original RegNetX takes in 3x224x224 input images and generates Nx7x7
#       feature maps before the decoder, this modified RegNetX will take in
#       3x56x56 input images and generate Nx7x7 feature maps before the decoder.
#       For reference, an implementation of this network took ~ 112 s per epoch
#       for training, validation and checkpoint saving on Sep 27, 2020 using a
#       free GPU runtime in Google Colab.
#
################################################################################

################################################################################
#
# IMPORT
#
################################################################################

# torch
import torch
import torch.nn       as     nn
import torch.optim    as     optim
from   torch.autograd import Function
import torch.nn.functional as F

# torch utils
import torchvision
import torchvision.transforms as transforms

# additional libraries
import os
import urllib.request
import zipfile
import time
import math
import numpy             as np
import matplotlib.pyplot as plt

################################################################################
#
# PARAMETERS
#
################################################################################

# data
DATA_DIR_1        = 'data'
DATA_DIR_2        = 'data/imagenet64'
DATA_DIR_TRAIN    = 'data/imagenet64/train'
DATA_DIR_TEST     = 'data/imagenet64/val'
DATA_FILE_TRAIN_1 = 'Train1.zip'
DATA_FILE_TRAIN_2 = 'Train2.zip'
DATA_FILE_TRAIN_3 = 'Train3.zip'
DATA_FILE_TRAIN_4 = 'Train4.zip'
DATA_FILE_TRAIN_5 = 'Train5.zip'
DATA_FILE_TEST_1  = 'Val1.zip'
DATA_URL_TRAIN_1  = 'https://github.com/arthurredfern/UT-Dallas-CS-6301-CNNs/raw/master/Data/Train1.zip'
DATA_URL_TRAIN_2  = 'https://github.com/arthurredfern/UT-Dallas-CS-6301-CNNs/raw/master/Data/Train2.zip'
DATA_URL_TRAIN_3  = 'https://github.com/arthurredfern/UT-Dallas-CS-6301-CNNs/raw/master/Data/Train3.zip'
DATA_URL_TRAIN_4  = 'https://github.com/arthurredfern/UT-Dallas-CS-6301-CNNs/raw/master/Data/Train4.zip'
DATA_URL_TRAIN_5  = 'https://github.com/arthurredfern/UT-Dallas-CS-6301-CNNs/raw/master/Data/Train5.zip'
DATA_URL_TEST_1   = 'https://github.com/arthurredfern/UT-Dallas-CS-6301-CNNs/raw/master/Data/Val1.zip'
DATA_BATCH_SIZE   = 512
DATA_NUM_WORKERS  = 4
DATA_NUM_CHANNELS = 3
DATA_NUM_CLASSES  = 100
DATA_RESIZE       = 64
DATA_CROP         = 56
DATA_MEAN         = (0.485, 0.456, 0.406)
DATA_STD_DEV      = (0.229, 0.224, 0.225)
D=13
w0=24
wa=36
wm=2.5
b=1
g=8
q=1
# model
# add model parameters here
def generate_parameters_regnet(D, w0, wa, wm, b, g, q):

  u = w0 + wa * np.arange(D) # Equation 1
  s = np.log(u / w0) / np.log(wm) # Equation 2

  s = np.round(s) #Rounding the possible block sizes s
  w = w0 * np.power(wm, s) # Equation 3
  w = np.round(w / 8) * 8 # Make all the width list divisible by 8

  w, d = np.unique(w.astype(np.int), return_counts=True) #Finding depth and width lists.

  gtemp = np.minimum(g, w//b)
  w = (np.round(w // b / gtemp) * gtemp).astype(int) #To make all the width compatible with group sizes of the 3x3 convolutional layers
  g = np.unique(gtemp // b)[0]
  return (w, d, b, g, q)

# training
# add training parameters here
TRAINING_LR_MAX          = 0.001
TRAINING_LR_INIT_SCALE   = 0.01
TRAINING_LR_INIT_EPOCHS  = 5
TRAINING_LR_FINAL_SCALE  = 0.01
TRAINING_LR_FINAL_EPOCHS = 55
# TRAINING_LR_FINAL_EPOCHS = 2 # uncomment for a quick test
TRAINING_NUM_EPOCHS      = TRAINING_LR_INIT_EPOCHS + TRAINING_LR_FINAL_EPOCHS
TRAINING_LR_INIT         = TRAINING_LR_MAX*TRAINING_LR_INIT_SCALE
TRAINING_LR_FINAL        = TRAINING_LR_MAX*TRAINING_LR_FINAL_SCALE
# file
# add file parameters here
# file
FILE_NAME = 'CifarResNetV2.pt'
FILE_SAVE = 0
FILE_LOAD = 0


################################################################################
#
# DATA
#
################################################################################

# create a local directory structure for data storage
if (os.path.exists(DATA_DIR_1) == False):
    os.mkdir(DATA_DIR_1)
if (os.path.exists(DATA_DIR_2) == False):
    os.mkdir(DATA_DIR_2)
if (os.path.exists(DATA_DIR_TRAIN) == False):
    os.mkdir(DATA_DIR_TRAIN)
if (os.path.exists(DATA_DIR_TEST) == False):
    os.mkdir(DATA_DIR_TEST)

# download data
if (os.path.exists(DATA_FILE_TRAIN_1) == False):
    urllib.request.urlretrieve(DATA_URL_TRAIN_1, DATA_FILE_TRAIN_1)
if (os.path.exists(DATA_FILE_TRAIN_2) == False):
    urllib.request.urlretrieve(DATA_URL_TRAIN_2, DATA_FILE_TRAIN_2)
if (os.path.exists(DATA_FILE_TRAIN_3) == False):
    urllib.request.urlretrieve(DATA_URL_TRAIN_3, DATA_FILE_TRAIN_3)
if (os.path.exists(DATA_FILE_TRAIN_4) == False):
    urllib.request.urlretrieve(DATA_URL_TRAIN_4, DATA_FILE_TRAIN_4)
if (os.path.exists(DATA_FILE_TRAIN_5) == False):
    urllib.request.urlretrieve(DATA_URL_TRAIN_5, DATA_FILE_TRAIN_5)
if (os.path.exists(DATA_FILE_TEST_1) == False):
    urllib.request.urlretrieve(DATA_URL_TEST_1, DATA_FILE_TEST_1)

# extract data
with zipfile.ZipFile(DATA_FILE_TRAIN_1, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR_TRAIN)
with zipfile.ZipFile(DATA_FILE_TRAIN_2, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR_TRAIN)
with zipfile.ZipFile(DATA_FILE_TRAIN_3, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR_TRAIN)
with zipfile.ZipFile(DATA_FILE_TRAIN_4, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR_TRAIN)
with zipfile.ZipFile(DATA_FILE_TRAIN_5, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR_TRAIN)
with zipfile.ZipFile(DATA_FILE_TEST_1, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR_TEST)

# transforms
transform_train = transforms.Compose([transforms.RandomResizedCrop(DATA_CROP), transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor(), transforms.Normalize(DATA_MEAN, DATA_STD_DEV)])
transform_test  = transforms.Compose([transforms.Resize(DATA_RESIZE), transforms.CenterCrop(DATA_CROP), transforms.ToTensor(), transforms.Normalize(DATA_MEAN, DATA_STD_DEV)])

# data sets
dataset_train = torchvision.datasets.ImageFolder(DATA_DIR_TRAIN, transform=transform_train)
dataset_test  = torchvision.datasets.ImageFolder(DATA_DIR_TEST,  transform=transform_test)

# data loader
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=DATA_BATCH_SIZE, shuffle=True,  num_workers=DATA_NUM_WORKERS, pin_memory=True, drop_last=True)
dataloader_test  = torch.utils.data.DataLoader(dataset_test,  batch_size=DATA_BATCH_SIZE, shuffle=False, num_workers=DATA_NUM_WORKERS, pin_memory=True, drop_last=True)

#
for data in dataloader_test:

        # extract a batch of data and move it to the appropriate device
  inputs, labels = data
  print(labels)

  


In [ ]:
################################################################################
#
# NETWORK BUILDING BLOCK
#
################################################################################

#Down sample
class Downsample(nn.Module):
  def __init__(self, in_filters, out_filters, stride):
    super(Downsample, self).__init__()

    self.conv1x1 = nn.Conv2d(in_filters, out_filters, kernel_size=1, stride=stride, bias=False)
    self.bn = nn.BatchNorm2d(out_filters)

  def forward(self, x):
    return self.bn(self.conv1x1(x))


#stem
class Stem(nn.Module):
  def __init__(self, No, Ni=3):
    super(Stem, self).__init__()

    self.conv_3x3 = nn.Conv2d(Ni, No, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn = nn.BatchNorm2d(No)

  def forward(self, X):
    return F.relu(self.bn(self.conv_3x3(X)))

#head

class Head(nn.Module):

  def __init__(self, Ni, classes):

    super(Head, self).__init__()

    self.avgpool = nn.AdaptiveAvgPool2d(output_size=1)
    self.fc = nn.Linear(Ni, classes)
      

  def forward(self, x):

      out = self.avgpool(x)
      out = torch.flatten(out, 1)
      out = self.fc(out)
      return out

class Layer(nn.Module):
  def __init__(self, in_filters, depth, width,  bottleneck_ratio, group_size, se_ratio,first_layer):
    super(Layer, self).__init__()
    
    self.layers = []

    #Total bottleneck blocks in a layer = Depth d
    for i in range(depth):
      if first_layer=="true":
        x_block = XBlock(in_filters, width, bottleneck_ratio, group_size, 1,se_ratio)
      else:
        stride = 2 if i == 0 else 1
        x_block = XBlock(in_filters, width, bottleneck_ratio, group_size, stride,se_ratio)

      self.layers.append(x_block)
      in_filters = width

    self.layers = nn.Sequential(*self.layers)
  
  def forward(self, x):
    out = self.layers(x)
    return out

#Ni in_filters
#No out_filters
#Fr bottleneck_ratio
#Sr Stride
#w width
#d depth
# X block
class XBlock(nn.Module):
  def __init__(self, in_filters, out_filters, bottleneck_ratio, group_size, stride=1, se_ratio=0):
    super(XBlock, self).__init__()

    #1x1 Bottleneck Convolution Block
    bottleneck_filters = in_filters // bottleneck_ratio
    self.conv1_1x1 = nn.Conv2d(in_filters, bottleneck_filters, kernel_size=1, bias=False) 
    self.bn1 = nn.BatchNorm2d(bottleneck_filters)

    #3x3 Convolution Block with Group Convolutions ---> ResNext alike structure
    num_groups = bottleneck_filters // group_size
    if stride==1:
      self.conv2_3x3 = nn.Conv2d(bottleneck_filters, bottleneck_filters, kernel_size=3, stride=1, padding=1, groups=num_groups, bias=False)
    elif stride==2:
      self.conv2_3x3 = nn.Conv2d(bottleneck_filters, bottleneck_filters, kernel_size=3, stride=2, padding=1, groups=num_groups, bias=False)

    self.bn2 = nn.BatchNorm2d(bottleneck_filters)


    #Downsample if stride=2
    self.downsample = Downsample(in_filters, out_filters, stride) if stride == 2 or in_filters != out_filters else None

    #1x1 Convolution Block
    self.conv3_1x1 = nn.Conv2d(bottleneck_filters, out_filters, kernel_size=1, bias=False)
    self.bn3 = nn.BatchNorm2d(out_filters)

  def forward(self, x):
    residual = x
    
    out = F.relu(self.bn1(self.conv1_1x1(x)))
    out = F.relu(self.bn2(self.conv2_3x3(out)))


    
    out = self.bn3(self.conv3_1x1(out))

    if self.downsample is not None:
      residual = self.downsample(x)
    
    out += residual
    out = F.relu(out)

    return out


In [ ]:

################################################################################
#
# NETWORK
#
################################################################################

# define
class Model(nn.Module):

    # initialization

     def __init__(self, data_num_channels, params, DATA_NUM_CLASSES):
        # parent initialization
      super(Model, self).__init__()
      self.in_filters = 32
      self.w, self.d, self.b, self.g, self.se_ratio = params
      self.num_layers = 4
      print(params)

            # add your code here
        # operations needed to create a modified RegNetX-200MF network
        # use the parameterized XBlock defined to simplify this section

      self.stem = Stem(self.in_filters)
      self.body = []

      for i in range(self.num_layers):
        if(i==0):
          layer = Layer(self.in_filters, self.d[i], self.w[i], self.b, self.g,self.se_ratio, first_layer="true")
        else:
          layer = Layer(self.in_filters, self.d[i], self.w[i], self.b, self.g, self.se_ratio, first_layer="false")

        self.body.append(layer)
        self.in_filters = self.w[i]
    
    #Body Part: Four Layers containing bottleneck residual blocks
      self.body = nn.Sequential(*self.body)

    #Head Part: Classification Step FC + AveragePool
      self.head = Head(self.w[-1], DATA_NUM_CLASSES)


    # forward path
     def forward(self, x):

        # add your code here
        # tie together the operations to create a modified RegNetX-200MF
        y=self.stem(x)
        y=self.body(y)
        y=self.head(y)
        # return
        return y
params = generate_parameters_regnet(D, w0, wa, wm, b, g,q)
# create
model = Model(DATA_NUM_CHANNELS,params,DATA_NUM_CLASSES)
# enable data parallelization for multi GPU systems
if (torch.cuda.device_count() > 1):
    model = nn.DataParallel(model)
print('Using {0:d} GPU(s)'.format(torch.cuda.device_count()), flush=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

# transfer the network to the device
model.to(device)


from torchsummary import summary
summary(model, input_size=(3, 56, 56))


In [ ]:
################################################################################
#
# ERROR AND OPTIMIZER
#
################################################################################
EPOCHS = 2

def lr_schedule(epoch):

    # linear warmup followed by cosine decay
    if epoch < TRAINING_LR_INIT_EPOCHS:
        lr = (TRAINING_LR_MAX - TRAINING_LR_INIT)*(float(epoch)/TRAINING_LR_INIT_EPOCHS) + TRAINING_LR_INIT
    else:
        lr = (TRAINING_LR_MAX - TRAINING_LR_FINAL)*max(0.0, math.cos(((float(epoch) - TRAINING_LR_INIT_EPOCHS)/(TRAINING_LR_FINAL_EPOCHS - 1.0))*(math.pi/2.0))) + TRAINING_LR_FINAL

    return lr
# error (softmax cross entropy)
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

# add your code here
# define the error criteria and optimizer

################################################################################
#
# TRAINING
#
################################################################################
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

# transfer the network to the device
model.to(device)
accuracy=[]
# model loading
if FILE_LOAD == 1:
    checkpoint = torch.load(FILE_NAME)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1


# add your code here
# perform network training, validation and checkpoint saving
# see previous examples in the Code directory
for epoch in range(EPOCHS):

    # initialize train set statistics
    model.train()
    training_loss = 0.0
    num_batches   = 0

    # set the learning rate for the epoch
    for g in optimizer.param_groups:
        g['lr'] = lr_schedule(epoch)

    # cycle through the train set
    for data in dataloader_train:

        # extract a batch of data and move it to the appropriate device
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward pass, loss, backward pass and weight update
        outputs = model(inputs)
        loss    = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # update statistics
        training_loss = training_loss + loss.item()
        num_batches   = num_batches + 1

    # initialize test set statistics
    model.eval()
    test_correct = 0
    test_total   = 0

    # no weight update / no gradient needed
    with torch.no_grad():

        # cycle through the test set
        for data in dataloader_test:

            # extract a batch of data and move it to the appropriate device
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # forward pass and prediction
            outputs      = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            # update test set statistics
            test_total   = test_total + labels.size(0)
            test_correct = test_correct + (predicted == labels).sum().item()
    # epoch statistics
            accuracy=(100.0*test_correct/test_total)
    print('Epoch {0:2d} lr = {1:8.6f} avg loss = {2:8.6f} accuracy = {3:5.2f}'.format(epoch, lr_schedule(epoch), (training_loss/num_batches)/DATA_BATCH_SIZE, (100.0*test_correct/test_total)))


In [ ]:
################################################################################
#
# TEST
#
################################################################################

# initialize test set statistics
model.eval()
test_correct = 0
test_total   = 0

# initialize class statistics
class_correct = list(0. for i in range(DATA_NUM_CLASSES))
class_total   = list(0. for i in range(DATA_NUM_CLASSES))

# no weight update / no gradient needed
with torch.no_grad():

    # cycle through the test set
    for data in dataloader_test:

        # extract a batch of data and move it to the appropriate device
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # forward pass and prediction
        outputs      = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        # update test set statistics
        test_total   = test_total + labels.size(0)
        test_correct = test_correct + (predicted == labels).sum().item()

        # update class statistics
        c = (predicted == labels).squeeze()
        for i in range(labels.size(0)):
            label                 = labels[i]
            class_correct[label] += c[i].item()
            class_total[label]   += 1

# test set statistics
print('Accuracy of test set = {0:5.2f}'.format((100.0*test_correct/test_total)))
print('')


DATA_CLASS_NAMES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

for i in range(10):
    print('Accuracy of {0:5s}    = {1:5.2f}'.format(DATA_CLASS_NAMES[i], (100.0*class_correct[i]/class_total[i])))
